In [35]:
import pickle
from revolve2.modular_robot import ModularRobot
from revolve2.modular_robot.brain.cpg import BrainCpgNSEnvironmentalControl
from revolve2.modular_robot.brain.cpg._make_cpg_network_structure_neighbor import (
    active_hinges_to_cpg_network_structure_neighbor,
)
from revolve2.modular_robot_physical import Config

import math
import numpy as np
from revolve2.modular_robot.body.v1 import *
from revolve2.modular_robot.body import RightAngles
from revolve2.modular_robot.body.base import ActiveHinge
from revolve2.modular_robot_physical import UUIDKey

from revolve2.modular_robot_physical.remote import test_physical_robot

In [36]:
CONTROL_FREQUENCY = 30 # 20
RUN_DURATION = 120

In [41]:
def darwinism() -> BodyV1:
    body = BodyV1()
    body.core_v1.front = ActiveHingeV1(RightAngles.DEG_90)
    body.core_v1.front.attachment = ActiveHingeV1(RightAngles.DEG_270)
    body.core_v1.front.attachment.attachment = ActiveHingeV1(RightAngles.DEG_0)

    body.core_v1.right = ActiveHingeV1(RightAngles.DEG_90)
    body.core_v1.right.attachment = ActiveHingeV1(RightAngles.DEG_270)
    body.core_v1.right.attachment.attachment = ActiveHingeV1(RightAngles.DEG_0)

    body.core_v1.back = ActiveHingeV1(RightAngles.DEG_90)
    body.core_v1.back.attachment = ActiveHingeV1(RightAngles.DEG_270)
    body.core_v1.back.attachment.attachment = ActiveHingeV1(RightAngles.DEG_0)

    return body


def lamarckism() -> BodyV1:
    body = BodyV1()
    body.core_v1.front = ActiveHingeV1(RightAngles.DEG_0)
    body.core_v1.front.attachment = ActiveHingeV1(RightAngles.DEG_0)
    body.core_v1.front.attachment.attachment = ActiveHingeV1(RightAngles.DEG_0)

    body.core_v1.back = ActiveHingeV1(RightAngles.DEG_90)
    body.core_v1.back.attachment = ActiveHingeV1(RightAngles.DEG_270)

    body.core_v1.right = ActiveHingeV1(RightAngles.DEG_90)
    body.core_v1.right.attachment = ActiveHingeV1(RightAngles.DEG_270)

    body.core_v1.left = ActiveHingeV1(RightAngles.DEG_90)
    body.core_v1.left.attachment = ActiveHingeV1(RightAngles.DEG_270)
    return body

## Configure Darwinism


In [42]:
with open("brains_pickle/darw_brain.pickle", "rb") as f:
    brain_params = pickle.load(f)

In [43]:
body = darwinism()
hinges = body.find_modules_of_type(ActiveHinge)
pins = [17,27,23,10,25,11,7,6,13]

hinge_mapping = {UUIDKey(hinge): pin for hinge, pin in zip(hinges, pins)}
initial_hinge_positions = {UUIDKey(hinge): 0.0 for hinge in hinges}
inverse_servos = {10: True, 7: True}

initial_state, _, weight_matrix, _ = brain_params
print(initial_state)

weight_matrix = weight_matrix / np.max(np.abs(weight_matrix))
(
    cpg_network_structure,
    output_mapping,
) = active_hinges_to_cpg_network_structure_neighbor(hinges)
# initial_state = cpg_network_structure.make_uniform_state(0.5 * math.sqrt(2))

brain = BrainCpgNSEnvironmentalControl(
    initial_state=initial_state,
    weight_matrix=weight_matrix,
    output_mapping=output_mapping,
)

robot = ModularRobot(body=body, brain=brain)

[0.78539816 0.78539816 0.78539816 0.78539816 0.78539816 0.78539816
 0.78539816 0.78539816 0.78539816 0.78539816 0.78539816 0.78539816
 0.78539816 0.78539816 0.78539816 0.78539816 0.78539816 0.78539816]


In [44]:
config = Config(
    modular_robot=robot,
    hinge_mapping=hinge_mapping,
    initial_hinge_positions=initial_hinge_positions,
    run_duration=RUN_DURATION,
    control_frequency=CONTROL_FREQUENCY,
    inverse_servos=inverse_servos,
)

with open("darwinism_conf.pickle", "wb") as f:
    pickle.dump(config, f)